## Megathon '23
# Team: CyberSpace
### Problem Statement 1
by _Govt. of Telangana_

In [270]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
from PIL import Image
import numpy as np
import cv2
from tensorflow import keras
import imageio

In [271]:
# Define a global variable to store the file path
selected_file_path = None

def browse_image():
    global selected_file_path
    file_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.png *.jpg *.jpeg *.gif *.bmp *.ppm *.pgm")])
    if file_path:

        # Display the selected image
        img = Image.open(file_path)
        img = img.resize((200, 200))  # Resize the image to fit your window
        img = ImageTk.PhotoImage(img)
        image_display.config(image=img)
        image_display.image = img  # Keep a reference to avoid garbage collection

        # Set the global variable to the selected file path
        selected_file_path = file_path
        
        # ANALYZING
        if(field_type()):   
            if(paddy_or_not()):
                analyze_type()
                analyze_variety()



## Main Application

In [272]:
# Create the main application window
app = tk.Tk()
app.title("Analysing Paddy Cultivation")

# app.geometry("500x250")
description_label = tk.Label(app, text="Description:")
description_label.pack()

desc="We have attempted to address this problem in a stagewise, divide and conquer fashion.\n Firstly, given a satellite view of a geographical area, we aim to notify whether there is scope of paddy cultivation or not. Secondly, we look at the probable areas flagged by our first model. The second model is trained to differentiate between paddy and other crops.Lastly, we have models to detect the health of paddy and identify the variety of paddy being cultivated."

description_text = tk.Label(app, text=desc, wraplength=450)
description_text.pack()

In [273]:
# Create and pack a button to browse and select an image
browse_button = tk.Button(app, text="Browse Image", command=browse_image)
browse_button.pack()

image_display = tk.Label(app)
image_display.pack()
image_display.place()

In [274]:
predicted_field=tk.Label(app, text="Is field: ")
predicted_field.pack()

In [275]:
predicted_ispaddy=tk.Label(app, text="Is paddy: ")
predicted_ispaddy.pack()

In [276]:
predicted_type=tk.Label(app, text="Analyzed Type: None")
predicted_type.pack()

In [277]:
predicted_vari=tk.Label(app, text="Variety of paddy: None")
predicted_vari.pack()

## Predicting

In [278]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import tensorflow as tf
import numpy as np

### 1. To check if it is a field

In [279]:
def field_type():
    global selected_file_path
    model = tf.keras.models.load_model('field_type.h5')

    img = cv2.imread(selected_file_path)
    img = cv2.resize(img, (128, 128))
    img = img / 255.0
    img = np.expand_dims(img, axis=0)
    prediction = model.predict(img)
    prob = prediction[0][0] + prediction[0][2] + prediction[0][6]
    if prob > 0.5:
        predicted_field.config(text="Is field: True")
        return True
    else:
        predicted_field.config(text="Is field: False")
        return False

### 2. To check if it is paddy

In [280]:
def paddy_or_not():
    global selected_file_path
    model = tf.keras.models.load_model('paddy_or_not.h5')

    image = load_img(selected_file_path, target_size=(224,224))
    image = img_to_array(image)
    image = image / 255.0  # Normalize the image
    image = tf.expand_dims(image, 0)  # Add a batch dimension

    result = model.predict(image)

    class_indices = {0: 'jute', 1: 'maize', 2: 'rice', 3: 'sugarcane', 4: 'wheat'}
    predicted_class = class_indices[np.argmax(result)]
    if predicted_class == 'rice':
        predicted_ispaddy.config(text="Is paddy: "+"True")
        return True  # If the model predicts the image as rice
    else:
        predicted_ispaddy.config(text="Is paddy: "+"False")
        return False

    return True

### 3. To find the type of paddy

In [281]:
def analyze_type():
    global selected_file_path
    model = tf.keras.models.load_model('paddy-type_classification.h5')

    image = load_img(selected_file_path, target_size=(256,256))
    image = img_to_array(image)
    image = image / 255.0  # Normalize the image
    image = tf.expand_dims(image, 0)  # Add a batch dimension

    predicted = model.predict(image)

    types= ['Healthy', 'Mild', 'Moderate', 'Severe']
    y_pred = [np.array(pred).argmax() for pred in predicted]

    predicted_type.config(text="Analyzed Type: "+ types[y_pred[0]])

### 4. To detect the varierty of paddy

In [282]:
def analyze_variety():
    global selected_file_path
    model = tf.keras.models.load_model('rice_variety_model.h5')

    image = load_img(selected_file_path, target_size=(150,150))
    image = img_to_array(image)
    image = image / 255.0  # Normalize the image
    image = tf.expand_dims(image, 0)  # Add a batch dimension

    predicted = model.predict(image)

    types= ['asd16', 'ADT_46', 'ADT-44', 'Muttakar rice', 'ADT39', 'MDU-3', 'ADT48', 'BPT5204', 'TPS3', 'mdu5', 'PMK4', 'IR20', 'IR_64', 'CORH 4', 'CORH3', 'MDU-4', 'CO50', 'TPS4', 'co46', 'Kuliparichan rice', 'adt37', 'ADT43_1', 'Sivappu chitrakar rice', 'PY-4', 'Norungan', 'ADT 49', 'TRY3', 'ADT38', 'TRY-1', 'CORH-1', 'CO47', 'CR1009', 'TPS-2', 'IR_36', 'bhavani', 'CO48', 'ADT47', 'CORH-2', 'asd20', 'ADT45', 'CO49', 'CO43', 'whiteponni', 'ADTRH-1', 'asd17', 'ADT36', 'RMD1']
    y_pred = [np.array(pred).argmax() for pred in predicted]
    predicted_vari.config(text="Variety of paddy: "+ types[y_pred[0]])


In [283]:
# Start the Tkinter main loop
app.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\drshw\anaconda3\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\drshw\AppData\Local\Temp/ipykernel_10164/1726156782.py", line 20, in browse_image
    if(field_type()):
  File "C:\Users\drshw\AppData\Local\Temp/ipykernel_10164/860324818.py", line 3, in field_type
    model = tf.keras.models.load_model('field_type.h5')
  File "c:\Users\drshw\anaconda3\lib\site-packages\keras\src\saving\saving_api.py", line 238, in load_model
    return legacy_sm_saving_lib.load_model(
  File "c:\Users\drshw\anaconda3\lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "c:\Users\drshw\anaconda3\lib\site-packages\keras\src\saving\legacy\save.py", line 234, in load_model
    raise IOError(
OSError: No file or directory found at field_type.h5


In [284]:
print("Selected file path:", selected_file_path)

Selected file path: C:/Users/drshw/Desktop/Megathon '23/Paddy-Onfield-Images/Severe/Severe (1).JPG
